# EEG-to-Speech 파이프라인 워크플로우

이 노트북은 EEG 데이터에서 음성을 생성하는 전체 파이프라인의 워크플로우를 보여줍니다. 데이터 다운로드부터 모델 학습, 평가, 결과 시각화까지의 전체 과정을 단계별로 설명합니다.

## 목차
1. 환경 설정 및 라이브러리 설치
2. 데이터 다운로드 및 준비
3. 데이터 탐색 및 시각화
4. 모델 구성 요소 초기화
5. 모델 학습
6. 모델 평가
7. 결과 시각화 및 분석
8. 결론 및 향후 연구 방향

In [1]:
import torch
print(torch.cuda.is_available())  # True가 출력되어야 정상 작동
print(torch.cuda.device_count())  # 사용 가능한 GPU 개수
print(torch.cuda.get_device_name(0))  # GPU 이름 출력


True
1
NVIDIA GeForce RTX 4060


## 1. 환경 설정 및 라이브러리 설치

필요한 라이브러리를 설치하고 기본 환경을 설정합니다.

In [2]:
# 필요한 라이브러리 설치
!pip install torch torchaudio numpy matplotlib scipy scikit-learn mne librosa soundfile tqdm transformers diffusers accelerate

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# 기본 라이브러리 임포트
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import json
import time
from tqdm import tqdm
from IPython.display import Audio, display

# 랜덤 시드 설정
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 장치: {device}")

사용 장치: cuda


In [4]:
# 프로젝트 디렉토리 설정
PROJECT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
CODE_DIR = os.path.join(PROJECT_DIR, 'code')
DATA_DIR = os.path.join(PROJECT_DIR, 'data')
RESULTS_DIR = os.path.join(PROJECT_DIR, 'results')
MODELS_DIR = os.path.join(PROJECT_DIR, 'models')
PRETRAINED_CACHE_DIR = os.path.join(PROJECT_DIR, 'pretrained_models')

# 디렉토리 생성
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(PRETRAINED_CACHE_DIR, exist_ok=True)

# 코드 디렉토리를 시스템 경로에 추가
if CODE_DIR not in sys.path:
    sys.path.append(CODE_DIR)

print(f"프로젝트 디렉토리: {PROJECT_DIR}")
print(f"코드 디렉토리: {CODE_DIR}")
print(f"데이터 디렉토리: {DATA_DIR}")
print(f"결과 디렉토리: {RESULTS_DIR}")
print(f"모델 디렉토리: {MODELS_DIR}")
print(f"사전 학습된 모델 캐시 디렉토리: {PRETRAINED_CACHE_DIR}")

프로젝트 디렉토리: C:\Users\user\Desktop\jm\eeg2speech_experiment\eeg2speech
코드 디렉토리: C:\Users\user\Desktop\jm\eeg2speech_experiment\eeg2speech\code
데이터 디렉토리: C:\Users\user\Desktop\jm\eeg2speech_experiment\eeg2speech\data
결과 디렉토리: C:\Users\user\Desktop\jm\eeg2speech_experiment\eeg2speech\results
모델 디렉토리: C:\Users\user\Desktop\jm\eeg2speech_experiment\eeg2speech\models
사전 학습된 모델 캐시 디렉토리: C:\Users\user\Desktop\jm\eeg2speech_experiment\eeg2speech\pretrained_models


## 2. 데이터 다운로드 및 준비

Inner Speech 데이터셋과 LibriSpeech 데이터셋을 다운로드하고 준비합니다.

In [5]:
# 데이터 다운로드 모듈 임포트
from data_download import InnerSpeechDownloader, LibriSpeechDownloader

# Inner Speech 데이터셋 다운로드
inner_speech_dir = os.path.join(DATA_DIR, 'inner_speech')
inner_speech_downloader = InnerSpeechDownloader(inner_speech_dir)
inner_speech_downloader.download()

Inner Speech 데이터셋 다운로드 중...
이미 다운로드된 저장소가 있습니다: C:\Users\user\Desktop\jm\eeg2speech_experiment\eeg2speech\data\inner_speech\Inner_Speech_Dataset
저장소를 최신 버전으로 업데이트했습니다.
OpenNeuro 데이터셋은 용량이 크므로 자동으로 다운로드하지 않습니다.
필요한 경우 다음 URL에서 수동으로 다운로드하세요: https://openneuro.org/datasets/ds003626/versions/1.0.2


'C:\\Users\\user\\Desktop\\jm\\eeg2speech_experiment\\eeg2speech\\data\\inner_speech\\Inner_Speech_Dataset'

In [6]:
# LibriSpeech 데이터셋 다운로드 (일부만 다운로드)
librispeech_dir = os.path.join(DATA_DIR, 'librispeech')
librispeech_downloader = LibriSpeechDownloader(librispeech_dir)
librispeech_downloader.download(subset='dev-clean')

LibriSpeech dev-clean 데이터셋 다운로드 중...
dev-clean 다운로드 중...


330007KB [00:35, 9228.32KB/s]                                                                        


dev-clean 압축 해제 중...
LibriSpeech dev-clean 데이터셋 준비 완료: C:\Users\user\Desktop\jm\eeg2speech_experiment\eeg2speech\data\librispeech\dev-clean


'C:\\Users\\user\\Desktop\\jm\\eeg2speech_experiment\\eeg2speech\\data\\librispeech\\dev-clean'

## 3. 데이터 탐색 및 시각화

다운로드한 데이터셋을 탐색하고 시각화합니다.

In [10]:
# 데이터 전처리 모듈 임포트
from data_preprocessing import EEGProcessor, AudioProcessor, EEGAudioDataset

# EEG 프로세서 생성
eeg_processor = EEGProcessor(data_dir=inner_speech_dir)

# EEG 데이터 로드
eeg_data = eeg_processor.load_data()

# EEG 데이터 정보 출력
print(f"EEG 데이터 형태: {eeg_data['eeg'].shape}")
print(f"EEG 채널 수: {eeg_data['channels'].shape[0]}")
print(f"EEG 샘플 수: {len(eeg_data['events'])}")
print(f"EEG 이벤트 유형: {set(eeg_data['event_types'])}")

AttributeError: 'EEGProcessor' object has no attribute 'load_data'

In [11]:
# EEG 데이터 시각화
eeg_processor.visualize_eeg(eeg_data, sample_idx=0, n_channels=10, title="EEG 데이터 샘플")

NameError: name 'eeg_data' is not defined

In [12]:
# 오디오 프로세서 생성
audio_processor = AudioProcessor(data_dir=librispeech_dir)

# 오디오 파일 로드
audio_files = audio_processor.get_audio_files()

# 오디오 파일 정보 출력
print(f"오디오 파일 수: {len(audio_files)}")
print(f"오디오 파일 예시: {audio_files[:5]}")

C:\Users\user\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

C:\Users\user\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--laion--clap-htsat-fused. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

경고: C:\Users\user\Desktop\jm\eeg2speech_experiment\eeg2speech\data\librispeech\librispeech\dev-clean 디렉토리를 찾을 수 없습니다.
오디오 파일 수: 0
오디오 파일 예시: []


In [13]:
# 오디오 데이터 시각화
audio_processor.visualize_audio(audio_files[0], title="오디오 데이터 샘플")

# 오디오 재생
waveform, sample_rate = audio_processor.load_audio(audio_files[0])
display(Audio(waveform, rate=sample_rate))

AttributeError: 'AudioProcessor' object has no attribute 'visualize_audio'

In [14]:
# 데이터셋 생성
dataset = EEGAudioDataset(
    eeg_data=eeg_data,
    audio_files=audio_files,
    eeg_processor=eeg_processor,
    audio_processor=audio_processor
)

print(f"데이터셋 크기: {len(dataset)}")

NameError: name 'eeg_data' is not defined

In [15]:
# 데이터셋 샘플 확인
sample = dataset[0]
print(f"EEG 데이터 형태: {sample['eeg'].shape}")
print(f"오디오 데이터 형태: {sample['audio'].shape}")
print(f"멜 스펙트로그램 형태: {sample['mel_spectrogram'].shape}")

NameError: name 'dataset' is not defined

## 4. 모델 구성 요소 초기화

EEG 인코더, 잠재 벡터 매핑 모델, 사전 학습된 모델을 초기화합니다.

In [16]:
# 모델 모듈 임포트
from eeg_encoder import create_eeg_encoder
from latent_mapping import create_latent_mapping_model
from pretrained_models import create_pretrained_model_manager, PretrainedModelIntegrator
from eeg2speech_trainer import create_eeg2speech_trainer
from eeg2speech_pipeline import EEG2SpeechPipeline

C:\ProgramData\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
C:\ProgramData\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


RuntimeError: Failed to import diffusers.pipelines.audioldm2.pipeline_audioldm2 because of the following error (look up to see its traceback):
cannot import name 'VitsModel' from 'transformers' (C:\ProgramData\anaconda3\Lib\site-packages\transformers\__init__.py)

In [17]:
# 설정 정의
config = {
    'eeg': {
        'n_channels': 128,  # EEG 채널 수
        'n_times': 2000,    # EEG 시간 포인트 수
        'vae_latent_dim': 512,  # VAE 잠재 공간 차원 (AudioLDM2와 호환)
        'clap_latent_dim': 512,  # CLAP 잠재 공간 차원 (CLAP와 호환)
        'hidden_dims': [64, 128, 256, 512]  # 은닉층 차원
    },
    'mapping': {
        'mapping_type': 'mlp',  # 매핑 모델 유형 ('mlp', 'resnet', 'adversarial')
        'hidden_dims': [1024, 1024, 512],  # 은닉층 차원
        'dropout': 0.2  # 드롭아웃 비율
    },
    'training': {
        'batch_size': 32,  # 배치 크기
        'val_split': 0.2,  # 검증 세트 비율
        'test_split': 0.1,  # 테스트 세트 비율
        'random_seed': 42,  # 랜덤 시드
        'encoder_epochs': 100,  # EEG 인코더 에폭 수
        'mapping_epochs': 100,  # 잠재 벡터 매핑 모델 에폭 수
        'encoder_lr': 1e-4,  # EEG 인코더 학습률
        'mapping_lr': 1e-4,  # 잠재 벡터 매핑 모델 학습률
        'patience': 10  # 조기 종료 인내심
    },
    'generation': {
        'prompt': 'speech, clear voice',  # 텍스트 프롬프트
        'num_inference_steps': 50,  # 추론 단계 수
        'audio_length_in_s': 5.0  # 생성할 오디오 길이 (초)
    },
    'paths': {
        'data_dir': DATA_DIR,  # 데이터 디렉토리
        'output_dir': RESULTS_DIR,  # 결과 디렉토리
        'model_dir': MODELS_DIR,  # 모델 디렉토리
        'pretrained_cache_dir': PRETRAINED_CACHE_DIR  # 사전 학습된 모델 캐시 디렉토리
    }
}

# 설정 저장
config_path = os.path.join(PROJECT_DIR, 'config.json')
with open(config_path, 'w') as f:
    json.dump(config, f, indent=4)

print(f"설정 저장 완료: {config_path}")

설정 저장 완료: C:\Users\user\Desktop\jm\eeg2speech_experiment\eeg2speech\config.json


In [18]:
# EEG2Speech 파이프라인 생성
pipeline = EEG2SpeechPipeline(config_path=config_path, device=device)

print("EEG2Speech 파이프라인 생성 완료")

NameError: name 'EEG2SpeechPipeline' is not defined

## 5. 모델 학습

EEG 인코더와 잠재 벡터 매핑 모델을 학습합니다.

In [19]:
# 데이터셋 분할
train_dataset, val_dataset, test_dataset = pipeline.split_dataset(dataset)

print(f"학습 데이터셋 크기: {len(train_dataset)}")
print(f"검증 데이터셋 크기: {len(val_dataset)}")
print(f"테스트 데이터셋 크기: {len(test_dataset)}")

NameError: name 'pipeline' is not defined

In [20]:
# EEG 인코더 학습
encoder_history = pipeline.train_eeg_encoder(train_dataset, val_dataset)

# 학습 곡선 시각화
pipeline.visualize_training_history(encoder_history, title="EEG 인코더 학습 곡선")

NameError: name 'pipeline' is not defined

In [21]:
# 잠재 벡터 매핑 모델 학습
mapping_history = pipeline.train_latent_mapping(train_dataset, val_dataset)

# 학습 곡선 시각화
pipeline.visualize_training_history(mapping_history, title="잠재 벡터 매핑 모델 학습 곡선")

NameError: name 'pipeline' is not defined

## 6. 모델 평가

학습된 모델을 테스트 데이터셋에서 평가합니다.

In [22]:
# 모델 평가
evaluation_results = pipeline.evaluate(test_dataset)

# 평가 결과 출력
print("평가 결과:")
for metric, value in evaluation_results.items():
    print(f"{metric}: {value:.4f}")

NameError: name 'pipeline' is not defined

In [23]:
# 잠재 공간 시각화
pipeline.visualize_latent_space(test_dataset, title="잠재 공간 시각화")

NameError: name 'pipeline' is not defined

## 7. 결과 시각화 및 분석

생성된 오디오 샘플을 시각화하고 분석합니다.

In [24]:
# 테스트 샘플에서 오디오 생성
sample_idx = 0
sample = test_dataset[sample_idx]
eeg_data = sample['eeg'].unsqueeze(0).to(device)

# 오디오 생성
generated_audio, sample_rate = pipeline.generate_audio_from_eeg(eeg_data)

# 생성된 오디오 재생
display(Audio(generated_audio.cpu().numpy(), rate=sample_rate))

NameError: name 'test_dataset' is not defined

In [25]:
# 원본 오디오와 생성된 오디오 비교
original_audio = sample['audio'].cpu().numpy()
generated_audio = generated_audio.cpu().numpy()

plt.figure(figsize=(12, 6))

plt.subplot(2, 1, 1)
plt.title("원본 오디오 파형")
plt.plot(original_audio)

plt.subplot(2, 1, 2)
plt.title("생성된 오디오 파형")
plt.plot(generated_audio)

plt.tight_layout()
plt.show()

NameError: name 'sample' is not defined

In [26]:
# 여러 샘플에 대한 오디오 생성 및 저장
num_samples = 5
for i in range(min(num_samples, len(test_dataset))):
    sample = test_dataset[i]
    eeg_data = sample['eeg'].unsqueeze(0).to(device)
    
    # 오디오 생성
    generated_audio, sample_rate = pipeline.generate_audio_from_eeg(eeg_data)
    
    # 오디오 저장
    output_path = os.path.join(RESULTS_DIR, f"generated_audio_{i}.wav")
    pipeline.save_audio(generated_audio, sample_rate, output_path)
    
    print(f"샘플 {i}의 생성된 오디오 저장 완료: {output_path}")

NameError: name 'test_dataset' is not defined

## 8. 결론 및 향후 연구 방향

프로젝트의 결론과 향후 연구 방향을 정리합니다.

In [27]:
# 모델 저장
pipeline.save_model(os.path.join(MODELS_DIR, "eeg2speech_model.pt"))
print("모델 저장 완료")

NameError: name 'pipeline' is not defined

### 결론

이 프로젝트에서는 EEG 데이터에서 음성을 생성하는 파이프라인을 구현했습니다. 주요 구성 요소는 다음과 같습니다:

1. **이중 경로 EEG 인코더**: EEG 데이터를 잠재 표현으로 변환
2. **잠재 벡터 매핑 모델**: EEG 잠재 벡터를 오디오 잠재 벡터로 매핑
3. **사전 학습된 모델 활용**: AudioLDM2, CLAP, Whisper 등의 사전 학습된 모델 활용

### 향후 연구 방향

1. **더 큰 데이터셋 활용**: 더 많은 EEG 데이터와 오디오 데이터를 활용하여 모델 성능 향상
2. **실시간 처리**: 실시간 EEG 신호에서 음성을 생성하는 시스템 개발
3. **개인화**: 개인별 특성을 반영한 맞춤형 모델 개발
4. **다양한 언어 지원**: 다양한 언어에 대한 지원 확장
5. **하이브리드 접근 방식**: EEG와 다른 생체 신호를 결합한 하이브리드 접근 방식 탐구